<a href="https://colab.research.google.com/github/priyank96/idl-spring-22-project-deepar/blob/main/DL_192Vectors_Group_Data_WeightedLoss_Debayan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
! rm -rf /content/idl-spring-22-project-deepar
!git clone https://github.com/priyank96/idl-spring-22-project-deepar

Cloning into 'idl-spring-22-project-deepar'...
remote: Enumerating objects: 1836, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 1836 (delta 0), reused 0 (delta 0), pack-reused 1829
Receiving objects: 100% (1836/1836), 82.45 MiB | 28.30 MiB/s, done.
Resolving deltas: 100% (877/877), done.
Checking out files: 100% (1945/1945), done.


In [10]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [11]:
! mkdir -p '/content/drive/MyDrive/DeepARExperiments/'

In [ ]:
!pip install wandb
!wandb login

In [ ]:
import wandb

wandb.init(project="vector-training", entity="idl-deepar-spring-22", name="baseline-more-data-weighte-loss")

wandb: Currently logged in as: mmml (use `wandb login --relogin` to force relogin)


In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import logging
from sklearn import preprocessing
import torch.optim as optim
from tqdm import tqdm
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
!python3 /content/idl-spring-22-project-deepar/src/data/company_random_groups.py -gs=5 -r=50000 -w=192
! pip install kaleido

In [ ]:
!python3 /content/idl-spring-22-project-deepar/src/data/visualize.py

In [5]:
import pickle
with open('/content/idl-spring-22-project-deepar/data/index_to_company.pkl','rb') as f:
  index_to_company = pickle.load(f)

with open('/content/idl-spring-22-project-deepar/data/company_to_index.pkl','rb') as f:
  company_to_index = pickle.load(f)

with open('/content/idl-spring-22-project-deepar/data/sector_company_map.pkl','rb') as f:
  sector_company_map = pickle.load(f)

In [7]:
print(index_to_company)
print(company_to_index)
print(sector_company_map)
print(sector_company_map.keys())
print(sector_company_map['Technology'])

{0: 'FOMC', 1: 'CS', 2: 'UPST', 3: 'DEFTF', 4: 'PPGH', 5: 'SAVA', 6: 'ACAD', 7: 'IPA', 8: 'ARWR', 9: 'TGGI', 10: 'SNPW', 11: 'GRPN', 12: 'TWLO', 13: 'SPOT', 14: 'XNET', 15: 'SHAK', 16: 'LOTZ', 17: 'RKUNY', 18: 'CROX', 19: 'GRIL', 20: 'LGIH', 21: 'WKHS', 22: 'CCL', 23: 'WISH', 24: 'EBAY', 25: 'RDFN', 26: 'AMT', 27: 'OPEN', 28: 'PLD', 29: 'AHT', 30: 'PSLV', 31: 'CRKR', 32: 'CFG', 33: 'NU', 34: 'SI', 35: 'MVIS', 36: 'AVLR', 37: 'CSCO', 38: 'FSLY', 39: 'PLAN', 40: 'GNRC', 41: 'J', 42: 'GGG', 43: 'ASTR', 44: 'CAR', 45: 'B', 46: 'IDEX', 47: 'AERC', 48: 'ATVK', 49: 'UAVS', 50: 'ES', 51: 'PCG', 52: 'AGR', 53: 'HLGN', 54: 'D', 55: 'E', 56: 'XOM', 57: 'ET', 58: 'CVE', 59: 'DNN', 60: 'RUN', 61: 'DIDI', 62: 'U', 63: 'DLO', 64: 'OPRA', 65: 'DBMM', 66: 'FUBO', 67: 'ICOA', 68: 'CABO', 69: 'GM', 70: 'DKS', 71: 'ONON', 72: 'JMIA', 73: 'FND', 74: 'WM', 75: 'AJRD', 76: 'BLNK', 77: 'PL', 78: 'AMC', 79: 'EA', 80: 'SNAP', 81: 'OEG', 82: 'QS', 83: 'NKE', 84: 'YETI', 85: 'CPNG', 86: 'AMZN', 87: 'ENS', 88: 'TR

In [10]:
DATA_PATH = '/content/idl-spring-22-project-deepar/data'



stock_inputs = np.load(DATA_PATH + '/stock_inputs.npy', allow_pickle=True)
stock_labels = np.load(DATA_PATH + '/stock_labels.npy', allow_pickle=True)

stock_test_inputs = np.load(DATA_PATH + '/stock_test_inputs.npy',allow_pickle=True)
stock_test_labels = np.load(DATA_PATH + '/stock_test_labels.npy',allow_pickle=True)

In [ ]:
NUM_TRAIN_SAMPLES = 50254
NUM_TEST_SAMPLES  = 15704

stock_inputs_trimmed = stock_inputs[:NUM_TRAIN_SAMPLES,:,:]
stock_labels_trimmed = stock_labels[:NUM_TRAIN_SAMPLES,:]

stock_test_inputs_trimmed = stock_test_inputs[:NUM_TEST_SAMPLES,:,:]
stock_test_labels_trimmed = stock_test_labels[:NUM_TEST_SAMPLES,:]

In [ ]:
print('stock_inputs_trimmed shape', stock_inputs_trimmed.shape)
print('stock_labels_trimmed shape', stock_labels_trimmed.shape)
print('stock_test_inputs_trimmed shape', stock_test_inputs_trimmed.shape)
print('stock_test_labels_trimmed shape', stock_test_labels_trimmed.shape)

stock_inputs_trimmed shape (50254, 192, 13)
stock_labels_trimmed shape (50254, 192, 13)
stock_test_inputs_trimmed shape (15704, 192, 13)
stock_test_labels_trimmed shape (15704, 192, 13)


In [ ]:


def generate_horizon_weights(horizon_len):

  weight = [1]
  for i in range(1, horizon_len):
    weight.append(weight[-1]*2)

  weight_arr = np.array(weight)

  return weight_arr/ np.sum(weight_arr)

HORIZON_PERIOD = 5

params = {
    # 'num_classes': len(train_comp_ids),
    # 'embedding_dim':5,
    #  month, day, 5 companies open and volume
    'cov_dim': 7,
    'op_dim': 5,  # op does not have the "time covariates"
    'lstm_hidden_dim': 64,
    'lstm_layers':4,
    'window_size':192,
    'batch_size': 64,
    'learning_rate': 0.005,
    'epochs':100,
    'num_test_samples': NUM_TEST_SAMPLES,
    'num_train_samples': NUM_TRAIN_SAMPLES,
    'conditioning_period': 168,
    'prediction_period': 24,
    'num_companies': 5,
    'mu_sigma_dim': 1,  # each datapoint is one-d,
    'scheduler_step_size': 20,
    'scheduler_gamma': 0.95,
    'horizon_period': HORIZON_PERIOD,
    'horizon_weights': generate_horizon_weights(HORIZON_PERIOD)
}
wandb.config = params
print(params)

{'cov_dim': 7, 'op_dim': 5, 'lstm_hidden_dim': 64, 'lstm_layers': 4, 'window_size': 192, 'batch_size': 64, 'learning_rate': 0.005, 'epochs': 100, 'num_test_samples': 15704, 'num_train_samples': 50254, 'conditioning_period': 168, 'prediction_period': 24, 'num_companies': 5, 'mu_sigma_dim': 1, 'scheduler_step_size': 20, 'scheduler_gamma': 0.95, 'horizon_period': 5, 'horizon_weights': array([0.03225806, 0.06451613, 0.12903226, 0.25806452, 0.51612903])}


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, Sampler
from pathlib import Path
import sys

DATA_PATH = '/content/idl-spring-22-project-deepar/data'


class TrainDataset(Dataset):
    def __init__(self):
        self.data = stock_inputs
        self.label = stock_labels
        self.train_len = self.data.shape[0]


    def __len__(self):
        return self.train_len
  
    def __getitem__(self, index):
        x1 = torch.from_numpy(self.data[index][:,[0,2,4,6,8,10,11]].astype(np.float32))
        x2 = torch.from_numpy(self.label[index][:,[0,2,4,6,8]].astype(np.float32))
        return x1, x2  

class TestDataset(Dataset):
    def __init__(self):
        self.data = stock_test_inputs
        self.label = stock_test_labels
        self.test_len = self.data.shape[0]
        
    def __len__(self):
        return self.test_len

    def __getitem__(self, index):
      x1 = torch.from_numpy(self.data[index][:,[0,2,4,6,8,10,11]].astype(np.float32))
      x2 = torch.from_numpy(self.label[index][:,[0,2,4,6,8]].astype(np.float32))
      return x1, x2  

In [ ]:
class TrainDatasetHorizon(Dataset):
    def __init__(self):
        self.data = stock_inputs
        self.label = stock_labels
        self.train_len = self.data.shape[0]
        self.horizon_period = params['horizon_period']


    def __len__(self):
        return self.train_len
  
    def __getitem__(self, index):
        x1 = torch.from_numpy(self.data[index][:,[0,2,4,6,8,10,11]].astype(np.float32))
        x2 = torch.from_numpy(self.label[index][:,[0,2,4,6,8]].astype(np.float32))
        
        x_shift = [None]*self.horizon_period
        x_shift = [torch.roll(x2, shifts=-1*i, dims=0)[:-1*(self.horizon_period - 1 )] for i in range(0, self.horizon_period)]

        
        return x1[:-1*(self.horizon_period - 1 ), :], x_shift  

In [ ]:
train_horizon = TrainDatasetHorizon()

In [ ]:
for x1, x2 in train_horizon:

    # print(x2)
    print("x1 shape", x1.shape)
    print("x2 shape", x2[0].shape)

    print("x2[0]", x2[0])

    print("\n")
    print("x2[4]", x2[4])



    break

In [ ]:
train_data = TrainDatasetHorizon()
test_data  = TestDataset()

train_loader = torch.utils.data.DataLoader(train_data, batch_size=params['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=params['batch_size'], shuffle=False)


In [ ]:
def rmse(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, _ = predictions.shape
  numerator = np.sqrt((1/(N)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

def ND(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, _ = predictions.shape
  numerator = np.sum(np.abs(labels - predictions))

  denominator = np.sum(np.abs(labels))
  result =numerator/denominator

  return result


# MODEL


In [ ]:
class Network(nn.Module):
    def __init__(self, params):
        '''
        We define a recurrent network that predicts the 
        future values of a time-dependent variable based on
        past inputs and covariates.
        '''
        super(Network, self).__init__()
        self.params = params
        
        self.lstm = nn.LSTM(input_size=params['cov_dim'],
                            hidden_size=params['lstm_hidden_dim'],
                            num_layers=params['lstm_layers'],
                            bias=True,
                            batch_first=True,
                          )

        self.price_mu = nn.Sequential(
                        nn.Linear(params['lstm_hidden_dim'], params['num_companies']),
                        nn.GELU()
                      )
        
        self.price_sigma = nn.Sequential(
                        nn.Linear(params['lstm_hidden_dim'], params['num_companies']),
                        nn.Softplus()
                      )
        

    def forward(self, x, h0_c0=None):
        '''
        Predict mu and sigma of the distribution for z_t.
        '''
        
        batch_size = x.shape[0]
        seq_len = x.shape[1]
        cov_dim = x.shape[2]

        assert cov_dim == params['cov_dim']
        assert batch_size <= params['batch_size']
                  
        out1, hn_cn = self.lstm(input=x, hx=h0_c0)
              
        out_price_mu = self.price_mu(out1)
        out_price_sigma = self.price_sigma(out1)
        
        return out_price_mu, out_price_sigma, hn_cn



In [ ]:
model = Network(params)

In [ ]:
def gauss_fit_and_sample(price_mu, price_sigma):
  window_len = price_mu.shape[1]
  batch_size = price_mu.shape[0]

  price_mu_reshape = price_mu.view(batch_size * window_len * params['num_companies'], params['mu_sigma_dim'])
  price_sigma_reshape = price_sigma.view(batch_size * window_len * params['num_companies'], params['mu_sigma_dim'])

  gauss_price = torch.distributions.Normal(price_mu_reshape, price_sigma_reshape)

  price_sampled_reshape = gauss_price.sample()
  price_sampled = price_sampled_reshape.view(batch_size, window_len, params['num_companies'])

  return price_sampled

In [ ]:
# Test Code Conditioning Period + Prediction Period


def validate(model):

  model.eval()


  conditionining_period = params['conditioning_period']
  prediction_period     = params['prediction_period']

  assert conditionining_period + prediction_period == params['window_size']

  with torch.no_grad():
      
    overall_price_mu = None
    # overall_volume_mu = None

    overall_price_sigma = None
    # overall_volume_sigma = None

    overall_price_label = None
    # overall_volume_label = None

    overall_price_pred = None
    # overall_volume_pred = None

    for i, (ip_covariate, op_label) in enumerate(test_loader):

      ip_covariate = ip_covariate.cuda()
      op_label = op_label.cuda()

      cond_ip = ip_covariate[:, 0:params['conditioning_period'], :]
      cond_op = op_label[:, 0:params['conditioning_period'], :]

      pred_ip = ip_covariate[:, params['conditioning_period']:, :]
      pred_op = op_label[:, params['conditioning_period']:, :]

      # Step One - Forward Pass : Conditioning Period.
      price_mu, price_sigma, ht_ct = model.forward(x=cond_ip, h0_c0=None)

      # Sample GMM and get PRICE/ volume
      price_sampled = gauss_fit_and_sample(price_mu, price_sigma)

      price_sampled_last = price_sampled[:, -1, :].unsqueeze(1)

      batch_price_mu = price_mu
      batch_price_sigma = price_sigma

      batch_price_pred = price_sampled

      for t in range(0, prediction_period):
        pred_cov_ip = pred_ip[:, t, :].unsqueeze(1)

        # pred_cov_ip[:, 0, 2:2 + params['num_companies']] = price_sampled_last[:, 0, :]
        pred_cov_ip[:, 0, 0:params['num_companies']] = price_sampled_last[:, 0, :]
        price_pred_mu, price_pred_sigma, ht_ct = model.forward(x=pred_cov_ip,h0_c0=ht_ct)
        price_sampled_last = gauss_fit_and_sample(price_pred_mu, price_pred_sigma)

        batch_price_mu = torch.cat((batch_price_mu, price_pred_mu), dim=1)
        batch_price_sigma = torch.cat((batch_price_sigma, price_pred_sigma), dim=1)        
        batch_price_pred = torch.cat((batch_price_pred, price_sampled_last), dim=1)

      overall_price_mu = batch_price_mu if overall_price_mu is None else torch.cat((overall_price_mu, batch_price_mu), dim=0)
      overall_price_sigma = batch_price_sigma if overall_price_sigma is None else torch.cat((overall_price_sigma, batch_price_sigma), dim=0)
      overall_price_pred = batch_price_pred if overall_price_pred is None else torch.cat((overall_price_pred, batch_price_pred), dim=0)
      overall_price_label = op_label[:, :, 0:params['num_companies']] if overall_price_label is None else torch.cat((overall_price_label, op_label[:, :, 0:params['num_companies']]), dim=0)
    
  
  return overall_price_pred,overall_price_label, overall_price_mu, overall_price_sigma


In [ ]:
model = Network(params)

model_version='vectorized_gaussian_weighted_loss_v1.pt'
epochs = 100

optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=params['scheduler_step_size'], gamma=params['scheduler_gamma'])

for epoch in range(0, epochs):

  batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
  model.train()
  model.cuda()

  total_loss = 0
  total_price_loss = 0
  total_volume_loss = 0

  if os.path.exists(f'/content/drive/MyDrive/DeepARExperiments/{model_version}'):
      # model.load_state_dict(torch.load(f'{SAVE_PATH}{EXP_TAG}/model_saved_epoch{epoch-1}.pt')) 

      checkpoint = torch.load(f'/content/drive/MyDrive/DeepARExperiments/{model_version}')
      model.load_state_dict(checkpoint['model_state_dict'])
      optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      epoch = checkpoint['epoch'] + 1
  
  for i, (ip_covariate, op_label_list) in enumerate(train_loader):

    optimizer.zero_grad()

    ip_covariate = ip_covariate.cuda()
    op_label_list     = [op_label.cuda() for op_label in op_label_list]

    
    out_price_mu, out_price_sigma, ht_ct = model.forward(x=ip_covariate)
    
    batch_size = out_price_mu.shape[0]
    seq_with_horizon = params['window_size'] - params['horizon_period'] + 1

    out_price_mu = out_price_mu.view(batch_size * seq_with_horizon * params['num_companies'], params['mu_sigma_dim'])
    out_price_sigma = out_price_sigma.view(batch_size * seq_with_horizon * params['num_companies'], params['mu_sigma_dim'])

    loss = 0

    for i, op_label in enumerate(op_label_list):

      op_label_price = op_label.view(batch_size * seq_with_horizon * params['num_companies'], params['mu_sigma_dim'])
      gauss_price =  torch.distributions.Normal(out_price_mu, out_price_sigma)   

      sample_loss =  -1 * torch.sum(gauss_price.log_prob(op_label_price)) / (batch_size * seq_with_horizon * params['num_companies'])
      loss += sample_loss * params['horizon_weights'][i]
    loss.backward()

    total_loss += float(loss)
    optimizer.step()

    batch_bar.set_postfix(
    loss="{:.04f}".format(float(total_loss / (i + 1))),
    lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))

    batch_bar.update() 

  batch_bar.close() # You need this to close the tqdm bar
  overall_price_pred, overall_price_label, overall_price_mu, overall_price_sigma = validate(model)
  scheduler.step()

  torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': total_loss/len(train_loader),
          },  f'/content/drive/MyDrive/DeepARExperiments/{model_version}')


  rmse_price = rmse(overall_price_pred.view(-1, 1), overall_price_label.view(-1, 1))
  nd_price = ND(overall_price_pred.view(-1, 1), overall_price_label.view(-1, 1))
  wandb.log({"loss":loss})
  print(f"Epoch {epoch}, price loss {loss}")

Epoch 0, price loss 0.14529459178447723


Epoch 1, price loss -0.45010972023010254


Epoch 2, price loss -0.09896358847618103


Epoch 3, price loss -0.4134294390678406


Epoch 4, price loss -0.5188876390457153


...

Epoch 34, price loss -0.8472074270248413


Epoch 35, price loss -0.6392410397529602


Epoch 36, price loss -0.9479904174804688


Epoch 37, price loss -0.825102686882019


Train:  59%|█████▉    | 486/820 [00:21<00:14, 23.16it/s, loss=-77.8654, lr=0.0047]

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

def plot_test_output_data(actual,predicted, window_id,cov_id):
  x=np.linspace(1, len(actual[window_id]), num=len(actual[window_id]))
  df = pd.DataFrame()
  df['actual'] = actual[window_id,:,cov_id]
  df['predicted'] = predicted[window_id,:,cov_id]
  

  fig = go.Figure()
  fig.add_vline(x=168)
  fig.add_traces(go.Scatter(x=x, y = df['actual'], mode = 'lines', name = 'Actual'))
  fig.add_traces(go.Scatter(x=x, y = df['predicted'], mode = 'lines', name = 'Predicted'))

  fig.update_xaxes(title_text='Time Samples')
  fig.update_yaxes(title_text='Normalized Stock Opening Price')
  fig.show()
  


In [ ]:
for i in range(5):
  plot_test_output_data(overall_price_label.detach().cpu(),overall_price_pred.detach().cpu(), window_id=6969, cov_id=i)